<a href="https://colab.research.google.com/github/blackbaba980/GameOfThronesPredictions/blob/main/code/GameOfThronesPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


import math
import time

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

random_seed = 7
#set the random seeds 
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)

epochs = 200
lr = 0.01
weight_decay = 5e-4
hidden_units = 16
dropout = 0.5

# Dataset Preprocessing


In [ ]:
#per season adjaceny matrix, features of every node

def load_data(season_no):

  #read the edge and node files
  dir = "./data/"
  edge_file = dir + "got-s" + str(season_no) + "-edges.csv"
  node_file = dir + "got-s" + str(season_no) + "-m-nodes.csv"

  edges = pd.read_csv(edge_file)
  nodes = pd.read_csv(node_file)
  
  
  #only get those edges where the nodes are present
  node_ids = nodes['CHARACTERS ID']
  node_ids_reverse = node_ids.to_dict()
  node_ids = {v: k for k, v in node_ids_reverse.items()}
  
  #remove unnecessary columns
  nodes = nodes.drop(['CHARACTERS ID'], axis=1).astype(np.float32)
  edges = edges.drop(['Season'], axis=1)
  
  #Adjacency matrix
  adj = np.zeros((len(nodes), len(nodes)), dtype='int')

  for i in range(len(edges)):
    if edges.iloc[i]['Source'] in node_ids and edges.iloc[i]['Target'] in node_ids:
      src_index = node_ids[edges.iloc[i]['Source']]
      dst_index = node_ids[edges.iloc[i]['Target']]
      adj[src_index][dst_index] = edges.iloc[i]['Weight']


  #convert the adjacency matrix to networkx graph
  rows, cols = np.where(adj != 0)
  edges = zip(rows.tolist(), cols.tolist())
  gr = nx.Graph()
  gr.add_edges_from(edges)

  return torch.from_numpy(nodes.to_numpy()), torch.from_numpy(adj), gr, node_ids
  #return nodes, edges



features, adj, graph, node_ids = load_data(1)


# Per Season Graph

# Generative Model

In [ ]:
class GraphConvolution(Module):
  def __init__(self, in_features, out_features, bias=True):
    super(GraphConvolution, self).__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.weight = Parameter(torch.FloatTensor(in_features, out_features))
    if bias:
        self.bias = Parameter(torch.FloatTensor(out_features))
    else:
        self.register_parameter('bias', None)
    self.reset_parameters()

  def reset_parameters(self):
    stdv = 1. / math.sqrt(self.weight.size(1))
    self.weight.data.uniform_(-stdv, stdv)
    if self.bias is not None:
        self.bias.data.uniform_(-stdv, stdv)

  def forward(self, input, adj):
    support = torch.mm(input, self.weight)
    output = torch.spmm(adj, support)
    if self.bias is not None:
        return output + self.bias
    else:
        return output

  def __repr__(self):
    return self.__class__.__name__ + ' (' \
            + str(self.in_features) + ' -> ' \
            + str(self.out_features) + ')'

In [ ]:
class Encoder(nn.Module):
  def __init__(self, nfeat, nhid, nclass, dropout, nlayers):
    super(Encoder, self).__init__()

    self.layers = nn.ParameterList()
    self.layers.append(GraphConvolution(nfeat, nhid))
    
    for l in range(1, nlayers - 1):
      self.layers.append(GraphConvolution(nhid, nhid))
    
    self.layers.append(GraphConvolution(nhid, nclass))
    
    self.dropout = dropout
    self.embeddings = []

  def forward(self, x, adj, save_embeddings = False):
    for l in range(len(self.layers) - 1):
      x = F.relu(self.layers[l](x, adj))
      if save_embeddings:
        self.embeddings.append(x.clone().detach())
      x = F.dropout(x, self.dropout, training=self.training)

    x = self.layers[-1](x, adj)
    if save_embeddings:
      self.embeddings.append(x.clone().detach())
    return F.log_softmax(x, dim=1)

In [ ]:
enc = Encoder(10, 20, 10, dropout,2)

In [ ]:
enc(features, adj)

RuntimeError: ignored

In [ ]:
adj.dtype

torch.int64

In [ ]:
features.dtype

torch.float32